# Research Analyzer - Multi-Analyst News Research System

This notebook implements a LangGraph-based research system that creates a team of specialized news analysts to conduct research on any topic and synthesize their findings into a comprehensive report.

## Phase 1: Setup & Environment

In this phase, we install dependencies and configure API keys.

In [ ]:
# Install required dependencies
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langchain_community langchain_core tavily-python python-dotenv pydantic

In [ ]:
import os
import getpass

def _set_env(var: str):
    """Set environment variable from user input if not already set."""
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Set OpenAI API Key
_set_env("OPENAI_API_KEY")

In [ ]:
# Set Tavily API Key for web search
_set_env("TAVILY_API_KEY")

In [ ]:
# Initialize the Language Model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)
print("✓ LLM initialized successfully!")

## Phase 2: Data Models

Define Pydantic models for NewsAnalyst, AnalystTeam, and SearchQuery.

In [ ]:
from typing import List
from pydantic import BaseModel, Field

class NewsAnalyst(BaseModel):
    """Model representing a specialized news analyst."""
    affiliation: str = Field(
        description="Primary affiliation of the analyst (e.g., News Network, Think Tank, Research Institute).",
    )
    name: str = Field(
        description="Name of the analyst."
    )
    role: str = Field(
        description="Role of the analyst (e.g., Political Correspondent, Tech Reporter, Economic Analyst).",
    )
    description: str = Field(
        description="Description of the analyst's focus, expertise, and analytical approach.",
    )
    
    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"

print("✓ NewsAnalyst model defined!")

In [ ]:
class AnalystTeam(BaseModel):
    """Model representing a team of news analysts."""
    analysts: List[NewsAnalyst] = Field(
        description="Team of news analysts with diverse specializations.",
    )

class SearchQuery(BaseModel):
    """Model for search query generation."""
    search_query: str = Field(None, description="Search query for news retrieval.")

print("✓ AnalystTeam and SearchQuery models defined!")

## Phase 3: State Definitions

Define TypedDict states for LangGraph workflows.

In [ ]:
import operator
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import MessagesState

class GenerateAnalystsState(TypedDict):
    """State for the analyst generation subgraph."""
    topic: str  # Topic to analyze
    max_analysts: int  # Number of analysts
    human_analyst_feedback: str  # Human feedback
    analysts: List[NewsAnalyst]  # Generated analysts

print("✓ GenerateAnalystsState defined!")

In [ ]:
class AnalysisState(MessagesState):
    """State for individual analyst research."""
    max_num_turns: int  # Number turns of conversation
    context: Annotated[list, operator.add]  # Source docs
    analyst: NewsAnalyst  # Analyst conducting analysis
    analysis: str  # Analysis transcript
    sections: Annotated[list, operator.add]  # Final sections for report

print("✓ AnalysisState defined!")

In [ ]:
class ResearchGraphState(TypedDict):
    """State for the main research graph."""
    topic: str
    max_analysts: int
    human_analyst_feedback: str
    analysts: List[NewsAnalyst]
    sections: Annotated[list, operator.add]
    introduction: str
    content: str
    conclusion: str
    final_report: str

print("✓ ResearchGraphState defined!")

## Phase 4: Prompt Templates

Define instruction templates for all LLM prompts.

In [ ]:
# Analyst Creation Instructions
ANALYST_INSTRUCTIONS = """You are tasked with creating a team of specialized news analysts. Follow these instructions:

1. Review the topic:
{topic}

2. Examine any editorial feedback for analyst creation:

{human_analyst_feedback}

3. Determine the most important perspectives for comprehensive news analysis of this topic.

4. Pick the top {max_analysts} perspectives.

5. Assign one analyst to each perspective with relevant expertise:
   - Examples: Political Analysis, Economic Impact, Social/Cultural Effects, Scientific/Tech Angle, International Relations, etc."""

print("✓ ANALYST_INSTRUCTIONS defined!")

In [ ]:
# Question Generation Instructions
QUESTION_INSTRUCTIONS = """You are a news analyst conducting research on {topic}.

Your goal is to gather specific, actionable insights about the topic.

1. Insightful: Find information that provides depth and context.

2. Specific: Include concrete data, quotes, names, and recent developments.

Your analytical focus: {goals}

Begin by introducing yourself, then pose your analytical questions.

Continue to drill down until you have comprehensive insights.

When satisfied with your analysis, conclude with: \"Analysis complete!\"

Stay in character throughout your response."""

print("✓ QUESTION_INSTRUCTIONS defined!")

In [ ]:
# Search Instructions
SEARCH_INSTRUCTIONS = """Generate a search query for recent news and information.

Focus on the latest developments, news, and data points relevant to the conversation.

The query should be specific and targeted for news sources."""

# Answer Instructions
ANSWER_INSTRUCTIONS = """You are a news information expert.

Analyst focus: {goals}

Answer the analyst's question using this context:

{context}

Guidelines:

1. Use only information from the provided context.

2. Include specific numbers, dates, and data points.

3. Cite sources using [1], [2], etc.

4. List sources at the end:

[1] Source 1
[2] Source 2"""

print("✓ SEARCH_INSTRUCTIONS and ANSWER_INSTRUCTIONS defined!")

In [ ]:
# Section Writer Instructions
SECTION_WRITER_INSTRUCTIONS = """You are a news report writer.

Create a concise, professional section based on analyst research.

1. Analyze the source documents containing analyst research.

2. Use this structure:
## {focus} (section title)

### Key Findings

### Analysis

### Sources

3. Analyst focus: {focus}

4. Key Findings:
- Highlight specific data points and numbers
- Include dates and timeframes
- Note significant trends or changes

5. Analysis:
- Explain implications
- Connect findings to the broader topic
- Maximum 300 words
- Use numbered sources [1], [2]

6. Sources:
- Include all sources used
- Format: [1] URL or source name
- One per line
"""

print("✓ SECTION_WRITER_INSTRUCTIONS defined!")

In [ ]:
# Report Writer Instructions
REPORT_WRITER_INSTRUCTIONS = """You are creating a comprehensive news report on:

{topic}

Your analyst team has completed their research.

Task:

1. Review all analyst sections
2. Identify key insights and themes
3. Synthesize into a cohesive narrative
4. Highlight actionable conclusions or future implications

Format:

1. Use markdown
2. No preamble
3. No sub-headings
4. Start with: ## News Analysis
5. Preserve all citations [1], [2], etc.
6. Create consolidated Sources section:

## Sources
[1] Source 1
[2] Source 2

Analyst sections:

{context}"""

print("✓ REPORT_WRITER_INSTRUCTIONS defined!")

In [ ]:
# Introduction Instructions
INTRODUCTION_INSTRUCTIONS = """You are writing an investment analysis report on {topic}

You have all report sections available.

Write a compelling introduction section.

No preamble.

Target 100 words.

Use markdown.

Create a compelling title with # header, then ## Introduction section.

Preview the key areas of analysis covered in the report.

Report sections: {formatted_str_sections}"""

# Conclusion Instructions
CONCLUSION_INSTRUCTIONS = """You are writing an investment analysis report on {topic}

You have all report sections available.

Write a conclusion section.

No preamble.

Target 100 words.

Use markdown.

Use ## Conclusion header.

Summarize the key investment insights and provide an overall perspective.

Report sections: {formatted_str_sections}"""

print("✓ INTRODUCTION_INSTRUCTIONS and CONCLUSION_INSTRUCTIONS defined!")